In [4]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

Завантаження даних з CSV файлів

In [5]:
data_path = "data"
activities = ["idle", "running", "stairs", "walking"]
df_list = []

for activity in activities:
    activity_path = os.path.join(data_path, activity)
    files = [f for f in os.listdir(activity_path) if f.endswith(".csv")]

    for file in files:
        file_path = os.path.join(activity_path, file)
        df = pd.read_csv(file_path)
        df["activity"] = activity
        df_list.append(df)

Об'єднання всіх даних в один DataFrame та розрахунок часових ознак (mean, std, min, max) для кожного запису

In [6]:
full_df = pd.concat(df_list, ignore_index=True)

numeric_cols = full_df.select_dtypes(include=['float64']).columns

full_df["mean"] = full_df[numeric_cols].mean(axis=1)
full_df["std"] = full_df[numeric_cols].std(axis=1)
full_df["min"] = full_df[numeric_cols].min(axis=1)
full_df["max"] = full_df[numeric_cols].max(axis=1)

Розділення на навчальний та тестовий набори

In [7]:
X = full_df.drop(["activity"], axis=1)
y = full_df["activity"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Видалення стовпців 'activity' перед розрахунком часових ознак

In [8]:
numeric_cols = X.select_dtypes(include=['float64']).columns
time_domain_features = ["mean", "std", "min", "max"]

for feature in time_domain_features:
    X[feature] = X[numeric_cols].apply(getattr, args=(feature,), axis=1)

Скалювання ознак

In [9]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Без виключення стовпців 'activity'
X_test_scaled = scaler.transform(X_test)

Модель SVM

In [10]:
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_scaled, y_train)
svm_predictions = svm_model.predict(X_test_scaled)

Модель випадкового лісу

In [11]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)
rf_predictions = rf_model.predict(X_test_scaled)

Оцінка моделей

In [12]:
svm_report = classification_report(y_test, svm_predictions, zero_division=1)
rf_report = classification_report(y_test, rf_predictions, zero_division=1)

Вивід результатів

In [13]:
print("SVM Classification Report:")
print(svm_report)

print("\nRandom Forest Classification Report:")
print(rf_report)

SVM Classification Report:
              precision    recall  f1-score   support

        idle       0.87      0.98      0.92      6203
     running       0.92      0.87      0.90     20485
      stairs       1.00      0.00      0.00      1022
     walking       0.78      0.88      0.83     11062

    accuracy                           0.87     38772
   macro avg       0.89      0.68      0.66     38772
weighted avg       0.88      0.87      0.86     38772


Random Forest Classification Report:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00      6203
     running       1.00      1.00      1.00     20485
      stairs       1.00      0.99      0.99      1022
     walking       1.00      1.00      1.00     11062

    accuracy                           1.00     38772
   macro avg       1.00      1.00      1.00     38772
weighted avg       1.00      1.00      1.00     38772

